<i><b>Public AI</b></i>
<br>
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 2. Convolution Neural Network**
# Section 4. Fully Convolutional Network Segmentation_16


In [ ]:
# If You use in Colab, You Should run this script
import os
if (not os.path.exists("./multicampus_segmentation_generator") and
    not "multicampus_segmentation_generator" in os.getcwd()):
    !git clone https://github.com/public-ai/multicampus_segmentation_generator.git
    os.chdir("./multicampus_segmentation_generator")

In [ ]:
from tensorflow.keras.layers import Layer, Conv2D, MaxPool2D, Conv2DTranspose, ZeroPadding2D, Concatenate
from tensorflow.keras.layers import Input, UpSampling2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.layers import Softmax, Add
from tensorflow.keras.layers import Lambda, concatenate
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K 
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt 
from keras.utils import np_utils
import numpy as np
import tensorflow as tf
import cv2
import sys 
sys.path.append('../')
from generator import fasion_mnist_generator

## 1. 논문 살펴보기

![Imgur](https://i.imgur.com/HEHu5TU.png)
- Referece: Fully Convolutional Networks for Semantic Segmentation 


### 2. Architecture 살펴보기 
![Imgur](https://i.imgur.com/aPq05oO.png)


- lower(fine) layer 에는 좀더 많은 위치 정보들이 들어있고 higher(coarse) layer 에는 많은 문맥 정보(contexture Info) 들이 들어 있습니다.

## 3. Upsampling 
![Imgur](https://i.imgur.com/R65YsR5.png)
 - Bilinear Interpolation 
 - Transpose Convolution 

In [ ]:
from sklearn.datasets import load_sample_images
dataset =load_sample_images()
sample_image = dataset.images[1]
print('Sample Image shape : {}'.format(sample_image.shape))
sample_image = np.expand_dims(sample_image, axis=0)


inputs = Input(shape=[427, 640, 3])

# Bilinear Interpolation 
bilinear_img = UpSampling2D()(inputs)
bilinear_model = Model(inputs, bilinear_img)
bilineared_img = bilinear_model.predict(sample_image)

# Transpose Convolution(Deconvolution)
deconv_img = Conv2DTranspose(filters=3, kernel_size=3, strides=2, padding='same')(inputs)
deconv_model= Model(inputs, deconv_img)
deconved_img = deconv_model.predict(sample_image)

fig, axes = plt.subplots(1,3)
fig.set_size_inches(30,10)
axes = np.array(axes).ravel()
axes[0].imshow(sample_image[0])
axes[0].set_title('original')
axes[1].imshow(bilineared_img[0]/255.)
axes[1].set_title('bilinear')
axes[2].imshow(deconved_img[0]/255.)
axes[2].set_title('Transconv')
plt.show()

## 4. FCN 구현

### Load Dataset

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fasion_mnist_generator(1000, 10)
print('Train Image shape : ', train_images.shape)
print('Train Labels shape : ', train_labels.shape)
print('Test Image shape : ', test_images.shape)
print('Test Labels shape : ', test_labels.shape)

In [ ]:
fig, axes = plt.subplots(1,2)
fig.set_size_inches((10,10))
sample_image = train_images[0]
axes[0].imshow(train_images[0])
axes[0].set_title('Image')

sample_mask = np.argmax(train_labels[0], axis=-1)
axes[1].imshow(sample_mask)
axes[1].set_title('Label')

plt.show()

### Build Model Architecture 

![Imgur](https://i.imgur.com/Y1gaZ6w.png)

In [ ]:
tf.reset_default_graph()

# inputs
# 위 그림에 만족하는 Convolution Neural Network를 구성해주세요
# fix me!

#  models
# block 1, Image shape (128, 128) -> (64, 64)
# 위 그림에 만족하는 Convolution Neural Network를 구성해주세요
# fix me!

# block 2 Output Shape : (64, 64) -> (32, 32)
# 위 그림에 만족하는 Convolution Neural Network를 구성해주세요
# fix me!

# block 3 Output Shape : (32, 32) -> (16, 16)
# 위 그림에 만족하는 Convolution Neural Network를 구성해주세요
# fix me!

# block 4 Output Shape : (16, 16) -> (8, 8)
# 위 그림에 만족하는 Convolution Neural Network를 구성해주세요
# fix me!

# FC Layer to 1x1 Conv2D (8, 8)
# 위 그림에 만족하는 Convolution Neural Network를 구성해주세요
# fix me!

# Upsampling Layer 
up_activation = None

# (8, 8) -> (128, 128)
# 위 그림에 만족하는 Convolution Neural Network를 구성해주세요
# fix me!

# 모델을 생성해주세요
# fix me!

In [ ]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
# compile & fitting 
# optimizer 로는 Nesterov Accelerated Gradient(NAG)를 사용해주세요
# learning_rate=0.005, decay=1e-6, momentum=0.9
# fix me!

In [ ]:
pred = model.predict(test_images[:100]/255.)
pred_cls = np.argmax(pred, axis=-1)

In [ ]:
fig, axes = plt.subplots(2,10)
ognl_axes = np.array(axes)[0, :].flatten()
pred_axes = np.array(axes)[1, :].flatten()
fig.set_size_inches(30,10)
for i in range(10):
    ognl_axes[i].imshow(test_images[i])
    pred_axes[i].imshow(pred_cls[i] == 0)

---
⊙ Copyright(c) 2020 by PublicAI. All rights reserved <br>
All pictures, codes, writings cannot be copied without permission. <br>
Writen by PAI(info@publicai.co.kr) <br>
last updated on 2020/01/4 <br>

---